In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

### 1. FORMATAÇÃO DO DATAFRAME

In [2]:
df = pd.read_csv("desafio_indicium_imdb.csv")

In [3]:
display(df)

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,995,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
995,996,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
996,997,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
997,998,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


In [5]:
df.describe()

,Unnamed: 0,IMDB_Rating,Meta_score,No_of_Votes
count,999.000000,999.000000,842.000000,9.990000e+02
mean,500.000000,7.947948,77.969121,2.716214e+05
std,288.530761,0.272290,12.383257,3.209126e+05
min,1.000000,7.600000,28.000000,2.508800e+04
25%,250.500000,7.700000,70.000000,5.547150e+04
50%,500.000000,7.900000,79.000000,1.383560e+05
75%,749.500000,8.100000,87.000000,3.731675e+05
max,999.000000,9.200000,100.000000,2.303232e+06


FORMATAÇÃO DE RUNTIME

In [6]:
df["Runtime"]=df["Runtime"].str.replace(" min", "")
df["Runtime"]=df["Runtime"].astype(int)

EXCLUSÃO DE LINHAS "GROSS" + FORMATAÇÃO

In [7]:
df.dropna(subset=['Gross'], inplace=True)
df['Gross'] = df['Gross'].str.replace(',', '')
df['Gross']=df["Gross"].astype(int)
#str opera ddentro de um texto

FORMATAÇÃO "Released_Year"

In [8]:
df["Released_Year"]=df['Released_Year'].replace("PG", "1995")
df["Released_Year"]=df["Released_Year"].astype(int)

#maior e menor ano
maior_ano=df["Released_Year"].max()
print(maior_ano)
menor_ano=df["Released_Year"].min()
print(menor_ano)

2019
1921


COMPLETANDO "Meta_Score" E "Certificate"

In [9]:
df['Meta_score'] = df.groupby('Genre')['Meta_score'].transform(lambda x: x.fillna(x.median()))
df['Meta_score'].fillna(df['Meta_score'].median(), inplace=True)

df['Certificate'].fillna(df['Certificate'].mode()[0], inplace=True)
df.info()
#mediana para meta_score e moda para certificate

<class 'pandas.core.frame.DataFrame'>
Index: 830 entries, 0 to 996
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     830 non-null    int64  
 1   Series_Title   830 non-null    object 
 2   Released_Year  830 non-null    int64  
 3   Certificate    830 non-null    object 
 4   Runtime        830 non-null    int64  
 5   Genre          830 non-null    object 
 6   IMDB_Rating    830 non-null    float64
 7   Overview       830 non-null    object 
 8   Meta_score     830 non-null    float64
 9   Director       830 non-null    object 
 10  Star1          830 non-null    object 
 11  Star2          830 non-null    object 
 12  Star3          830 non-null    object 
 13  Star4          830 non-null    object 
 14  No_of_Votes    830 non-null    int64  
 15  Gross          830 non-null    int64  
dtypes: float64(2), int64(5), object(9)
memory usage: 110.2+ KB


/tmp/ipython-input-1679708411.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Meta_score'].fillna(df['Meta_score'].median(), inplace=True)
/tmp/ipython-input-1679708411.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [10]:
df.describe()

,Unnamed: 0,Released_Year,Runtime,IMDB_Rating,Meta_score,No_of_Votes,Gross
count,830.000000,830.000000,830.000000,830.000000,830.000000,8.300000e+02,8.300000e+02
mean,503.184337,1993.919277,124.056627,7.945301,77.380120,3.128067e+05,6.808257e+07
std,291.338616,21.147811,27.510988,0.279446,12.088552,3.365523e+05,1.098076e+08
min,1.000000,1921.000000,45.000000,7.600000,28.000000,2.508800e+04,1.305000e+03
25%,249.250000,1984.000000,104.000000,7.700000,70.000000,7.140450e+04,3.245338e+06
50%,507.500000,2000.000000,120.000000,7.900000,78.000000,1.858500e+05,2.345744e+07
75%,757.750000,2010.000000,138.000000,8.100000,86.000000,4.434590e+05,8.087634e+07
max,997.000000,2019.000000,242.000000,9.200000,100.000000,2.303232e+06,9.366622e+08


In [11]:
teste = df["Gross"][1:20]
print(teste)

1     534858444
2      57300000
3       4360000
4     377845905
5     107928762
6      96898818
7     292576195
8      37030102
9     315544750
10    330252182
11      6100000
12    342551365
13    171479930
14     46836394
15    290475067
16    112000000
18     53367844
20    188020017
21      7563397
Name: Gross, dtype: int64


NOVA TABELA

In [12]:
df.to_csv("desafio_indicium_imdb2.csv")

### 2. EDA DO DATAFRAME

In [13]:
df = pd.read_csv("desafio_indicium_imdb2.csv")

ATORES QUE MAIS APARECEM NO DATAFRAME

In [14]:
atores=pd.concat([df['Star1'], df['Star2'], df['Star3'], df['Star4'] ])
top10_atores=atores.value_counts()
print(f"Top 10 atores que mais aparecem no dataframe: \n{top10_atores.head(10)}")

Top 10 atores que mais aparecem no dataframe: 
Robert De Niro       17
Tom Hanks            14
Al Pacino            13
Brad Pitt            12
Clint Eastwood       11
Leonardo DiCaprio    11
Christian Bale       11
Matt Damon           11
Denzel Washington     9
Johnny Depp           9
Name: count, dtype: int64


GÊNEROS QUE MAIS APARECEM NO DATAFRAME

In [15]:
generos = df['Genre'].str.split(', ')
gen_separados=df.assign(gen_ind=generos).explode('gen_ind')

top10_generos=gen_separados["gen_ind"].value_counts()

print(top10_generos.head(10))

gen_ind
Drama        596
Comedy       191
Crime        171
Adventure    170
Action       156
Thriller     110
Romance      107
Biography     96
Mystery       84
Animation     67
Name: count, dtype: int64


DIRETORES QUE MAIS APARECEM NO DATAFRAME

In [16]:
top20_diretores=df["Director"].value_counts()

print(f"Os diretores que mais aparecem nos filmes são: \n{top20_diretores.head(20)}")

Os diretores que mais aparecem nos filmes são: 
Director
Steven Spielberg     13
Martin Scorsese      10
Alfred Hitchcock      9
Woody Allen           8
Clint Eastwood        8
David Fincher         8
Quentin Tarantino     8
Christopher Nolan     8
Rob Reiner            7
Hayao Miyazaki        7
Billy Wilder          6
Joel Coen             6
Sergio Leone          6
Wes Anderson          6
Richard Linklater     6
Stanley Kubrick       6
Ridley Scott          6
Ron Howard            5
Alfonso Cuarón        5
Denis Villeneuve      5
Name: count, dtype: int64



MÉDIA DO IMDB_RATING DOS TOP20 DIRETORES

In [17]:
lista_diretores=top20_diretores.head(20).index
df_top20_dir=df[df["Director"].isin(lista_diretores)]
media_dir=df_top20_dir.groupby('Director')['IMDB_Rating'].mean().sort_values(ascending=False)

print(f"Média de IMDB por diretores do top 20:{media_dir}")

Média de IMDB por diretores do top 20:Director
Christopher Nolan    8.462500
Stanley Kubrick      8.266667
Sergio Leone         8.266667
Billy Wilder         8.200000
Quentin Tarantino    8.175000
Martin Scorsese      8.170000
Alfred Hitchcock     8.144444
Hayao Miyazaki       8.142857
Ridley Scott         8.083333
David Fincher        8.037500
Steven Spielberg     8.030769
Denis Villeneuve     7.980000
Ron Howard           7.920000
Clint Eastwood       7.912500
Richard Linklater    7.900000
Wes Anderson         7.833333
Rob Reiner           7.828571
Joel Coen            7.816667
Woody Allen          7.800000
Alfonso Cuarón       7.760000
Name: IMDB_Rating, dtype: float64


MELHORES NOTAS POR GÊNERO

In [18]:
notas_genero= gen_separados.groupby('gen_ind').agg(
    media_imdb=('IMDB_Rating', 'mean'),
    vezes_gen=('Series_Title', 'count')
).sort_values(by='media_imdb', ascending=False)

print(f"Média de Nota IMDB por gênero: \n{notas_genero.head(20)}")

#agg = agregar para média

Média de Nota IMDB por gênero: 
           media_imdb  vezes_gen
gen_ind                         
Western      8.037500         16
War          8.018919         37
Film-Noir    8.009091         11
Sci-Fi       7.991803         61
Mystery      7.969048         84
Adventure    7.962941        170
Crime        7.959649        171
Drama        7.951678        596
Action       7.946795        156
Romance      7.929907        107
Sport        7.926316         19
History      7.925581         43
Biography    7.925000         96
Animation    7.922388         67
Thriller     7.920909        110
Musical      7.914286         14
Music        7.909091         33
Fantasy      7.905357         56
Comedy       7.896859        191
Family       7.889583         48


MÉDIA DE MINUTOS DOS TOP 20 FILMES DE CADA GÊNERO

In [19]:
generos=df['Genre'].str.split(', ')
gen_separados = df.assign(gen_ind=generos).explode('gen_ind')

def media_minutos_top20(group):
    top20_filmes= group.nlargest(20, 'IMDB_Rating')
    return top20_filmes['Runtime'].mean()

duracao=gen_separados.groupby('gen_ind').apply(media_minutos_top20)
duracao_ordenada=duracao.sort_values(ascending=False)

print(f"Média de minutos dos top 20 filmes de cada gênero:{duracao_ordenada.head(21)}")

Média de minutos dos top 20 filmes de cada gênero:gen_ind
Drama        156.200000
History      155.550000
Biography    152.900000
Action       148.500000
Musical      145.571429
Adventure    143.700000
Crime        141.750000
Western      134.562500
Sci-Fi       134.450000
War          133.950000
Sport        132.631579
Mystery      126.450000
Thriller     125.550000
Music        122.600000
Romance      121.700000
Comedy       115.300000
Fantasy      114.500000
Family       112.600000
Horror       107.400000
Animation    104.900000
Film-Noir    101.363636
dtype: float64


/tmp/ipython-input-3794801104.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duracao=gen_separados.groupby('gen_ind').apply(media_minutos_top20)


MÉDIA DE VOTOS DO TOP 20 POR GÊNERO

In [20]:
generos=df['Genre'].str.split(', ')
gen_separados = df.assign(gen_ind=generos).explode('gen_ind')

def media_votos_top20(group):
  top_20_filmes = group.nlargest(20, "IMDB_Rating")
  return top_20_filmes["No_of_Votes"].mean()

votos=gen_separados.groupby('gen_ind').apply(media_votos_top20)
maiores_votos=votos.sort_values(ascending=False)

print(f"A média de votos dos top 20 filmes, considerando o gênero é de: {maiores_votos.head(21).to_string(float_format='{:.0f}'.format)} ")

A média de votos dos top 20 filmes, considerando o gênero é de: gen_ind
Drama       1319619
Action      1117516
Adventure   1060791
Crime       1012085
Sci-Fi       891018
Thriller     627323
Biography    542983
Animation    540956
Mystery      535346
Fantasy      534676
Comedy       419756
Romance      382811
War          350422
History      331524
Family       294828
Horror       279799
Western      272667
Sport        256619
Music        187786
Film-Noir     93989
Musical       85256 


/tmp/ipython-input-1481279033.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  votos=gen_separados.groupby('gen_ind').apply(media_votos_top20)


DÉCADAS QUE MAIS SE REPETEM NOS 100 MELHORES FILMES

In [21]:
df['Released_Year'] = pd.to_numeric(df['Released_Year'])

df['Decada']=(df['Released_Year'] // 10) * 10
melhores_100= df.head(100)
decadas= melhores_100['Decada'].value_counts()

print(decadas)


Decada
1990    23
2000    21
2010    20
1980    12
1970     9
1960     7
1950     4
1940     2
1930     2
Name: count, dtype: int64


### 3. MODELO DE MACHINE LEARNING IMDB

In [22]:
df=pd.read_csv("desafio_indicium_imdb2.csv")

In [23]:
from re import X
feature = ['Released_Year', 'Runtime', 'No_of_Votes', 'Gross', 'Meta_score', 'Genre', 'Director']
target = 'IMDB_Rating'

X=df[feature]
y=df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#colunas
numericas=["Released_Year", 'Runtime', 'No_of_Votes', 'Gross', 'Meta_score']
strings=['Genre', "Director"]
#pipeline
transofrmador_colunas=ColumnTransformer(
    transformers=[
        ('str', OneHotEncoder(handle_unknown='ignore'), strings)
    ],
    remainder='passthrough'
)

#Rodando
X_train_ok = transofrmador_colunas.fit_transform(X_train)
X_test_ok = transofrmador_colunas.transform(X_test)

print(f"Treino: {X_train_ok.shape}")
print(f"Teste: {X_test_ok.shape}")

Treino: (664, 579)
Teste: (166, 579)


TREINAMENTO

In [24]:
from sklearn.ensemble import RandomForestRegressor

modelo_imdb= RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

modelo_imdb.fit(X_train_ok, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

VALIDAÇÃO

In [25]:
from sklearn.metrics import mean_squared_error, r2_score

previsoes = modelo_imdb.predict(X_test_ok)

mse=mean_squared_error(y_test, previsoes)
r2=r2_score(y_test, previsoes)

print(f"Erro quadrático médio: {mse:.4f}")
print(f"Coeficiente de determinação: {r2:.4f}")

Erro quadrático médio: 0.0332
Coeficiente de determinação: 0.6095


PREVISÃO

In [26]:
filme_previsao = {
    'Released_Year': 1994,
    'Runtime': 142,
    'No_of_Votes': 2343110,
    'Gross': 28341469,
    'Meta_score': 80.0,
    'Genre': 'Drama',
    'Director': 'Frank Darabont'
}

df_dicionario=pd.DataFrame([filme_previsao])
preparador_filme=transofrmador_colunas.transform(df_dicionario)

previsao_nota=modelo_imdb.predict(preparador_filme)

print(f"A nota de IMDB prevista para o filme é: {previsao_nota[0]:.2f}")

A nota de IMDB prevista para o filme é: 8.77


SALVAR PKL

In [27]:
doc_pkl="modelo_imdb.pkl"
joblib.dump(modelo_imdb, doc_pkl)

['modelo_imdb.pkl']